In [2]:
import cv2
import dlib
import time

In [3]:
import math

In [4]:
cap = cv2.VideoCapture("./videos/TrafficVideo.mp4")
#cap = cv2.VideoCapture(0)

In [5]:
# estimates speed function for cars 
def estimate_speed(location1, location2):
    dist_in_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))

    ppm = 5
    dist_in_meters = dist_in_pixels / ppm
    fps = 28
    speed = dist_in_meters * fps * 3.6
    return speed


In [6]:
# import the Haar Cascade car classifier
car_classifier = cv2.CascadeClassifier("haarcascade_car.xml")

# resizing 
WIDTH = 1280
HEIGHT = 720 

frame_count = 0
currentCarID = 0
fps = 0

carTracker = {}
carNumbers = {}
carLocation1 = {}
carLocation2 = {}
speed = [None] * 1000



while True:
    start_time = time.time()
    ret, frame = cap.read()
    if type(frame) == type(None):
        break

    frame = cv2.resize(frame, (WIDTH, HEIGHT))
    result_frame = frame.copy()

    frame_count += 1
    delete_car_ids = []

    for carID in carTracker.keys():
        trackingQuality = carTracker[carID].update(frame)

        if trackingQuality < 8:
            delete_car_ids.append(carID)
        
    for carID in delete_car_ids:
        carTracker.pop(carID, None)
        carLocation1.pop(carID, None)
        carLocation2.pop(carID, None)
        
    if not (frame_count % 10):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cars = car_classifier.detectMultiScale(gray, 1.1, 13, 18, (24, 24))

        for (_x, _y, _w, _h) in cars:
            x = int(_x)
            y = int(_y)
            w = int(_w)
            h = int(_h)

            x_bar = x + 0.5 * w
            y_bar = y + 0.5 * h

            matchCarID = None

            for carID in carTracker.keys():
                trackedPosition = carTracker[carID].get_position()

                t_x = int(trackedPosition.left())
                t_y = int(trackedPosition.top())
                t_w = int(trackedPosition.width())
                t_h = int(trackedPosition.height())

                t_x_bar = t_x + 0.5 * t_w
                t_y_bar = t_y + 0.5 * t_h

                if ((t_x <= x_bar <= (t_x + t_w)) and (t_y <= y_bar <= (t_y + t_h)) and (x <= t_x_bar <= (x + w)) and (y <= t_y_bar <= (y + h))):
                    matchCarID = carID

            if matchCarID is None:

                tracker = dlib.correlation_tracker()
                tracker.start_track(frame, dlib.rectangle(x, y, x + w, y + h))

                carTracker[currentCarID] = tracker
                carLocation1[currentCarID] = [x, y, w, h]

                currentCarID = currentCarID + 1

    for carID in carTracker.keys():
        trackedPosition = carTracker[carID].get_position()

        t_x = int(trackedPosition.left())
        t_y = int(trackedPosition.top())
        t_w = int(trackedPosition.width())
        t_h = int(trackedPosition.height())

        cv2.rectangle(result_frame, (t_x, t_y), (t_x + t_w, t_y + t_h), (0,255,0), 4)
        carLocation2[carID] = [t_x, t_y, t_w, t_h]

    for i in carLocation1.keys():
        if frame_count >= 2:
            [x1, y1, w1, h1] = carLocation1[i]
            [x2, y2, w2, h2] = carLocation2[i]

            carLocation1[i] = [x2, y2, w2, h2]

            if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                
                if (speed[i] == None or speed[i] == 0) and y1 >= 30 and y1 <= 100:
                    speed[i] = estimate_speed([x1, y1, w1, h1], [x2, y2, w2, h2])

                if speed[i] != None:
                    cv2.putText(result_frame, str(int(speed[i])) + "km/h", (int(x1 + w1/2), int(y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 100), 2)

    cv2.imshow('frame', result_frame)

    if cv2.waitKey(1) == 27:
        break
    
    cv2.destroyAllWindows()


: 

In [1]:
#live video feed with functioning buttons pause and capture
import cv2
import tkinter as tk
from PIL import Image, ImageTk

class LiveVideoApp:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)

        # Loads pre-trained Haar Cascade classifier for car detection
        self.car_classifier = cv2.CascadeClassifier("haarcascade_car.xml")

        # Open the default camera for given device
        self.cap = cv2.VideoCapture(0)

        # Check if the camera is opened successfully, if not prints error message
        if not self.cap.isOpened():
            print("Error: Unable to open camera.")
            return

        # Create a canvas to display the live video feed in
        self.canvas = tk.Canvas(window, width=640, height=480)
        self.canvas.pack()

        # Adding functional buttons for controlling the live video feed - capture and pause
        self.btn_pause = tk.Button(window, text="Pause", width=10, command=self.pause_video)
        self.btn_pause.pack(pady=5)
        self.btn_capture = tk.Button(window, text="Capture Image", width=15, command=self.capture_image)
        self.btn_capture.pack(pady=5)

        # Start the live vide feed
        self.stream_video()

        self.window.mainloop()

    def stream_video(self):
        ret, frame = self.cap.read()

        if ret:
            # Converts the frame from OpenCV BGR format to PIL format
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)

            # Resize the image to fit the canvas
            img = img.resize((640, 480), Image.LANCZOS)


            # Converts the PIL image to Tkinter format and display it into the canvas
            self.photo = ImageTk.PhotoImage(image=img)
            self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)

            # Call the stream_video function recursively to continue streaming rather than stop
            self.window.after(10, self.stream_video)
        else:
            print("Error: Unable to read frame.")

    def pause_video(self):
        # enabling of pausing or resuming the live video feed 
        if self.btn_pause.cget("text") == "Pause":
            self.btn_pause.config(text="Resume")
            self.cap.release()
        else:
            self.btn_pause.config(text="Pause")
            self.cap = cv2.VideoCapture(0)
            self.stream_video()

    def capture_image(self):
        ret, frame = self.cap.read()

        if ret:
            # Save the captured image to directory
            cv2.imwrite('captured_image.png', frame)
            print("Image captured and saved as 'captured_image.png'.")
        else:
            print("Error: Unable to capture image.")

if __name__ == "__main__":
    root = tk.Tk()
    app = LiveVideoApp(root, "Live Video Feed")


Image captured and saved as 'captured_image.png'.
Error: Unable to read frame.


: 